In [ ]:
from epx import FREDJob, FREDModelConfig, SynthPop

from pathlib import Path
import os

import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

os.environ["FRED_PROJECT"] = str(Path.cwd())
path_to_fred_project = os.environ["FRED_PROJECT"]

In [ ]:
mpox_config = FREDModelConfig(synth_pop=SynthPop("US_2010.v5", ["Dane_County_WI"]),
    start_date = "2022-06-01",
    end_date = "2024-06-01")

fred_ref_files = {"data/bathhouse_transmission_MPOX_OUTBREAK.txt":"data/bathhouse_transmission_MPOX_OUTBREAK.txt"}
fred_include_files = [
    "model/main_monkeypox.fred",
    "model/mpox.fred",
    "model/msm.fred",
    "model/bathhouse.fred",
    "model/test_report.fred"]


mpox_job = FREDJob(
    config=[mpox_config],
    tags=["mpox_job","epx-client-1.2.0"],
    fred_version="11.0.1",
    fred_files=fred_include_files,
    ref_files=fred_ref_files
)

# Execute job
mpox_job.execute(300)

str(mpox_job.status)

In [ ]:
mpox_job.results.state(condition="BATHHOUSE", state="Visit", count_type="new")

In [ ]:
susceptible = mpox_job.results.state(
    condition="MPOX",
    state="Susceptible",
    count_type="new"
)

exposed = mpox_job.results.state(
    condition="MPOX",
    state="Exposed",
    count_type="new"
)

infectious = mpox_job.results.state(
    condition="MPOX",
    state="Infectious",
    count_type="new"
)


recovered = mpox_job.results.state(
    condition="MPOX",
    state="Recovered",
    count_type="new"
)

In [ ]:
susceptible

In [ ]:
susceptible.to_csv("sus_counts.csv")

In [ ]:
# rename the 'new' counts column to the name of the state
susceptible.rename(columns={'new':'Susceptible'},inplace=True)
exposed.rename(columns={'new':'Exposed'},inplace=True)
infectious.rename(columns={'new':'Infectious'},inplace=True)
recovered.rename(columns={'new':'Recovered'},inplace=True)

# build up the merged dataframe with counts for all states
all_states = pd.merge(susceptible,exposed)
all_states = pd.merge(all_states,infectious)
all_states = pd.merge(all_states,recovered)

#retrieve the sim_date from the column and add to the all_states frame.
all_states['sim_date']=mpox_job.results.dates().sim_date

In [ ]:
all_states.to_csv("all_mpox_data_vals1.csv")

In [ ]:
all_states['Exposed_7day_avg'] = all_states.Exposed.rolling(7).mean()
all_states['Recovered_7day_avg'] = all_states.Recovered.rolling(7).mean()
all_states['Infectious_7day_avg'] = all_states.Infectious.rolling(7).mean()

all_state_7day_df = all_states[~all_states['Exposed_7day_avg'].isnull()]

fig = px.line(
    all_state_7day_df,
    x='sim_date',
    y=['Exposed_7day_avg','Infectious_7day_avg','Recovered_7day_avg'], # pass a list to plot all states],
    title="Mpox outbreak in County")
fig.update_layout( # customize font and legend orientation & position
    font_family="Epistemix Label",
    yaxis_title="New infections per day, 7 day average",
    xaxis_title="Date",
    legend_title="State",
    title_font_size=30
)
fig.show()

In [ ]:
mpox_job.delete(interactive=False)